In [1]:
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
import datetime
import pandas as pd
import mysql.connector

In [2]:
klin_conn = {
    'host': 'localhost', #db in prod
    'user': 'root',
    'password': 'password',
    'database': 'klines_history',
    'port': "3306",
    'auth_plugin': 'mysql_native_password'
}


In [3]:
def sql_to_df(connector, table):    
    connexion = mysql.connector.connect(**connector)
    curseur = connexion.cursor()
    requete_sql = f"SELECT * FROM {table}"
    curseur.execute(requete_sql)
    dataframe = pd.DataFrame(curseur.fetchall(), columns=curseur.column_names)
    curseur.close()
    connexion.close()
    return dataframe

def get_result():
    df_klines = sql_to_df(klin_conn, 'klines')
    df_predictions = sql_to_df(klin_conn, 'predictions')

    df_work = df_predictions.merge(right = df_klines, on = ['Symbol', 'TimeStep', 'OpenTime'], how = 'left')
    df_work['precision'] = abs((df_work['ClosePrice_y'] - df_work['ClosePrice_x']) / df_work['ClosePrice_y']) * 100

    result = round(df_work['precision'].mean(),2)
    print('erreur moyenne calculee : ' + str(result) + '%')

In [4]:
get_result()

erreur moyenne calculee : 8.5%


In [5]:
df_klines = sql_to_df(klin_conn, 'klines')
df_predictions = sql_to_df(klin_conn, 'predictions')

df_work = df_predictions.merge(right = df_klines, on = ['Symbol', 'TimeStep', 'OpenTime'], how = 'left')
df_work['precision'] = abs((df_work['ClosePrice_y'] - df_work['ClosePrice_x']) / df_work['ClosePrice_y']) * 100

In [6]:
df_work

,Symbol,TimeStep,InTime,OpenTime,PctChange,ClosePrice_x,OpenPrice,HighPrice,LowPrice,ClosePrice_y,Volume,precision
0,ADAUSDT,MINUTELY,2023-09-19 19:23:27,2023-09-19 19:23:00,0.315008,0.336773,0.2558,0.2558,0.2558,0.2558,719.8,31.654977
1,ADAUSDT,MINUTELY,2023-09-19 19:23:27,2023-09-19 19:24:00,0.284101,0.328858,0.2557,0.2557,0.2557,0.2557,25742.6,28.610938
2,ADAUSDT,MINUTELY,2023-09-19 19:23:27,2023-09-19 19:25:00,0.291429,0.330348,0.2558,0.2558,0.2557,0.2557,1036.5,29.193437
3,ADAUSDT,MINUTELY,2023-09-19 19:23:27,2023-09-19 19:26:00,0.309009,0.334976,0.2557,0.2557,0.2556,0.2556,2509.8,31.054580
4,ADAUSDT,MINUTELY,2023-09-19 19:25:07,2023-09-19 19:27:00,0.103757,0.282341,0.2557,0.2557,0.2556,0.2557,7154.1,10.418866
...,...,...,...,...,...,...,...,...,...,...,...,...
65910,XRPUSDT,WEEKLY,2023-09-19 19:23:33,2023-09-25 02:00:00,0.084042,0.547116,0.5020,0.5075,0.4912,0.5020,935086300.0,8.987214
65911,XRPUSDT,WEEKLY,2023-09-19 19:23:33,2023-10-02 02:00:00,0.117457,0.555264,NaN,NaN,NaN,NaN,NaN,NaN
65912,XRPUSDT,WEEKLY,2023-09-19 19:23:33,2023-10-09 02:00:00,0.137530,0.560234,NaN,NaN,NaN,NaN,NaN,NaN
65913,XRPUSDT,WEEKLY,2023-09-19 19:23:33,2023-10-16 02:00:00,0.150155,0.588304,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
result = df_work[['Symbol', 'TimeStep', 'precision']].groupby(by=['Symbol', 'TimeStep']).mean()

In [8]:
result

precision
Symbol  TimeStep           
ADAUSDT DAILY     19.331652
        HOURLY    38.258406
        MINUTELY   9.226775
        WEEKLY    12.076026
BNBUSDT DAILY     17.355281
        HOURLY    33.112014
        MINUTELY   9.858332
        WEEKLY     6.037259
BTCUSDT DAILY      6.701504
        HOURLY    35.557310
        MINUTELY   7.142834
        WEEKLY    25.022424
ETHUSDT DAILY     10.251855
        HOURLY     5.852648
        MINUTELY   9.453495
        WEEKLY     1.674624
XRPUSDT DAILY     18.952038
        HOURLY    14.853405
        MINUTELY   5.395102
        WEEKLY     8.987214

In [63]:
result = result.reset_index()

In [67]:
print(result)

     Symbol  TimeStep  precision
0   ADAUSDT     DAILY  28.863272
1   ADAUSDT    HOURLY   9.182109
2   ADAUSDT  MINUTELY  19.232234
3   ADAUSDT    WEEKLY        NaN
4   BNBUSDT     DAILY  31.897039
5   BNBUSDT    HOURLY  19.169677
6   BNBUSDT  MINUTELY   7.258447
7   BNBUSDT    WEEKLY        NaN
8   BTCUSDT     DAILY   0.731397
9   BTCUSDT    HOURLY  24.723445
10  BTCUSDT  MINUTELY  22.226998
11  BTCUSDT    WEEKLY        NaN
12  ETHUSDT     DAILY  16.005708
13  ETHUSDT    HOURLY  27.369230
14  ETHUSDT  MINUTELY  23.955711
15  ETHUSDT    WEEKLY        NaN
16  XRPUSDT     DAILY  32.295646
17  XRPUSDT    HOURLY  22.716104
18  XRPUSDT  MINUTELY  18.714694
19  XRPUSDT    WEEKLY        NaN


In [ ]:
for symbol in result['Symbol'].unique():
    print()

In [ ]:
query = f"""
CREATE TABLE IF NOT EXISTS hist_detailed_precision (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date DATETIME,
    Symbol VARCHAR(255),
    TimeStep VARCHAR(255),
    Precision DOUBLE
)
"""

In [ ]:
query = f"""
CREATE TABLE IF NOT EXISTS hist_global_precision (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date DATETIME,
    Precision DOUBLE
)
"""